In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.stats import norm, probplot
import warnings
warnings.filterwarnings('ignore')

X_train = pd.read_csv('/home/yijie/kaggles/data/house_price/X_train.csv').to_numpy()
X_test = pd.read_csv('/home/yijie/kaggles/data/house_price/X_test.csv').to_numpy()
y_train = pd.read_csv('/home/yijie/kaggles/data/house_price/y_train.csv').to_numpy().squeeze()
X_train.shape, X_test.shape, y_train.shape

((1455, 393), (1459, 393), (1455,))

# Alternating Direction Method of Multipliers

[Info 1](https://zhuanlan.zhihu.com/p/106896627)

In [48]:
# Lasso
import numpy as np
import matplotlib.pyplot as plt

class Lasso:

    def __init__(self,X,y):
        '''X为纵列样本，横列特征
        loss = .5||y-Xbeta||^2+lambda*||beta||
        '''
        self.X = X
        self.y = y
        self.best_params = None # beta
        self.best_l1 = None # lambda
        self.best_loss = None

        # X = np.hstack((self.X,np.ones((self.X.shape[0],1),dtype=int)))
    
    def train_loss(self,params,l1):
        assert len(params) == self.X.shape[1], 'Mismatch between features and params number.'
        def MSE_(y,X,params):
            return np.sum([y[i]-(X@params)[i] for i in range(X.shape[0])])
        def L1_(l1,params):
            return l1*np.sum([np.abs(w) for w in params])
            
        return 1/2*MSE_(self.y,self.X,params) + L1_(l1,params)
    
    def predict(self,X_test,params=None,l1=None):
        if params is None and l1 is None:
            print("Best trained parameters are in-use.")
            params = self.best_params
            l1 = self.best_l1
        yp = X_test@params
        print("Loss with L1-penalty({0}) is: {1}".format(l1,self.loss(yp,params,l1)))
    
    def admm(self,l1,rho,threshold=1e-3):
        size = self.X.shape[1]
        
        beta, alpha, w = 0.1*np.ones((size,)), 0.2*np.ones((size,)), 0.3*np.ones((size,)),
        loss, table = 0, []
        X, y, XT, I = self.X, self.y, self.X.T, np.eye(size)
        # while loss>threshold:
        for iter in range(500):
            beta = np.linalg.inv(XT@X+rho*I)@(XT@y+rho*(alpha-w))
            mid = beta+w
            for i in range(size):
                if mid[i,]<(-l1/rho):
                    alpha[i,] = mid[i,]+l1/rho
                elif mid[i,]>l1/rho:
                    alpha[i,] = mid[i,]-l1/rho
                else:
                    alpha[i,] = 0
            w = w + beta -alpha

            loss = self.train_loss(beta,l1)
            table.append(loss)


            # # sparse strategy?
            # for param in beta:
            #     if param<1e-7:
            #         param = 0
        
        plt.figure()
        plt.style.use('ggplot')
        plt.title('Loss descent with admm')
        plt.plot(np.arange(len(table)),table)

        if (self.best_loss is None) or (loss<self.best_loss):
            self.loss = loss
            self.best_params = beta
            self.best_l1 = l1

        return 'Trained.'
                    
            

    def coordinate_descent(self):
        pass
    
    def param_init(self,size):
        return [1 for i in range(size)]

lasso = Lasso(X_train,y_train)

In [49]:
lasso.admm(l1=1,rho=1)

In [47]:
lasso.best_params

array([ 1.29875198e-02,  6.53396922e-06, -2.02474750e-06, -1.15971327e-05,
       -2.11826129e-04,  2.59066739e-03, -1.58704974e-03,  1.22956732e-02,
        2.11461348e-04,  3.48028117e-04, -1.01577241e-06, -4.14336946e-03,
       -8.20586856e-05,  2.85025093e-04, -8.77540732e-03,  5.44665579e-01,
        4.53195176e-04,  4.05958181e-01,  1.63325101e-02, -8.48876174e-04,
        1.03468117e-02, -1.04120865e-03,  1.85474886e-04,  2.41667151e-02,
        4.73060051e-04, -5.50562832e-06,  3.47536840e-06,  6.86079237e-03,
        3.33212929e-04,  1.81413941e-03, -1.41349642e-03,  1.85049070e-03,
        2.08067824e-02, -1.51071878e-05, -2.30308552e-03,  3.02036907e-04,
       -8.52025855e-04, -1.32973090e-06, -3.45454936e-03,  2.63787701e-05,
       -8.07047694e-06,  9.25438234e-03, -2.26695714e-04,  1.57102695e-04,
       -1.16218633e-04,  3.30253353e-03,  3.08668326e-06, -5.89683551e-08,
        2.12154791e-02, -5.48948164e-08, -2.41668605e-03,  2.95120084e-02,
       -8.52025943e-04,  

In [27]:
beta, alpha, w = 0.1*np.ones((393,)), 0.2*np.ones((393,)), 0.3*np.ones((393,)),
_ = np.linalg.inv(X_train.T@X_train+1*np.eye(X_train.shape[1]))@(X_train.T@y_train+1*(alpha-w))
_.shape

(393,)

In [20]:
(X_train.T@y_train).shape

(393,)

In [43]:
W = np.array([i*i for i in range(X_train.shape[1]+1)])
lasso.loss(W,1)

-307036297006.15674

In [16]:
(X_train.T@y_train).shape

(393,)

In [39]:
from sklearn.linear_model import Lasso
lasoo = Lasso()
lasoo.fit(X_train,y_train)
coef = lasoo.coef_
y_train, X_train[0,0:10]

(array([12.24769912, 12.10901644, 12.31717117, ..., 12.49313327,
        11.86446927, 11.90159023]),
 array([  4.18965474,   9.04204006,   7.        , 109.        ,
         53.        ,   5.28320373,   1.        ,   2.        ,
          1.        ,   1.09861229]))

In [46]:
# eigenvalue decomposition
M = np.matrix([[1,4,9],[4,9,16],[9,16,25]])
Sp,P = np.linalg.eigh(M)
D = np.array([[Sp[j]*(j==i) for j in range(3)] for i in range(3)])
D, Sp, P@D@P.T, P, P**-1, np.linalg.inv(M)

(array([[-2.58349038,  0.        ,  0.        ],
        [-0.        ,  0.08257361,  0.        ],
        [-0.        ,  0.        , 37.50091678]]),
 array([-2.58349038,  0.08257361, 37.50091678]),
 matrix([[ 1.,  4.,  9.],
         [ 4.,  9., 16.],
         [ 9., 16., 25.]]),
 matrix([[-0.77091878,  0.58208336, -0.2585792 ],
         [-0.40863669, -0.76341433, -0.50021457],
         [ 0.48856964,  0.27995985, -0.82639106]]),
 matrix([[-0.77091878, -0.40863669,  0.48856964],
         [ 0.58208336, -0.76341433,  0.27995985],
         [-0.2585792 , -0.50021457, -0.82639106]]),
 matrix([[ 3.875, -5.5  ,  2.125],
         [-5.5  ,  7.   , -2.5  ],
         [ 2.125, -2.5  ,  0.875]]))

In [44]:
None is None

True

In [18]:
X_train.T@X_train

array([[2.59936817e+04, 5.60039152e+04, 3.74703782e+04, ...,
        8.46612613e+01, 5.02794700e+03, 5.30489492e+02],
       [5.60039152e+04, 1.21040234e+05, 8.08635220e+04, ...,
        1.81840004e+02, 1.08792117e+04, 1.13022230e+03],
       [3.74703782e+04, 8.08635220e+04, 5.66890000e+04, ...,
        1.16000000e+02, 7.17200000e+03, 9.25000000e+02],
       ...,
       [8.46612613e+01, 1.81840004e+02, 1.16000000e+02, ...,
        2.00000000e+01, 0.00000000e+00, 0.00000000e+00],
       [5.02794700e+03, 1.08792117e+04, 7.17200000e+03, ...,
        0.00000000e+00, 1.19500000e+03, 0.00000000e+00],
       [5.30489492e+02, 1.13022230e+03, 9.25000000e+02, ...,
        0.00000000e+00, 0.00000000e+00, 1.23000000e+02]])

In [19]:
np.dot(X_train.T, X_train)

array([[2.59936817e+04, 5.60039152e+04, 3.74703782e+04, ...,
        8.46612613e+01, 5.02794700e+03, 5.30489492e+02],
       [5.60039152e+04, 1.21040234e+05, 8.08635220e+04, ...,
        1.81840004e+02, 1.08792117e+04, 1.13022230e+03],
       [3.74703782e+04, 8.08635220e+04, 5.66890000e+04, ...,
        1.16000000e+02, 7.17200000e+03, 9.25000000e+02],
       ...,
       [8.46612613e+01, 1.81840004e+02, 1.16000000e+02, ...,
        2.00000000e+01, 0.00000000e+00, 0.00000000e+00],
       [5.02794700e+03, 1.08792117e+04, 7.17200000e+03, ...,
        0.00000000e+00, 1.19500000e+03, 0.00000000e+00],
       [5.30489492e+02, 1.13022230e+03, 9.25000000e+02, ...,
        0.00000000e+00, 0.00000000e+00, 1.23000000e+02]])

In [40]:
coef

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00,  2.87138485e-05, -0.00000000e+00,
       -7.72927570e-05, -5.20345814e-05, -0.00000000e+00,  0.00000000e+00,
        2.29702514e-05,  0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
        0.00000000e+00, -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        7.86311756e-06, -0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
       -0.00000000e+00,  0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
       -0.00000000e+00,  0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        0.00000000e+00, -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
       -0.00000000e+00,  